<a href="https://colab.research.google.com/github/Ambrish001/Coastline_Monitoring/blob/main/Land_Cover_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
import imutils
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import glob
from tensorflow.keras.preprocessing import image
from skimage.metrics import hausdorff_distance as hdf
from skimage.metrics import structural_similarity as ssim

In [ ]:
!pip install sewar
from sewar.full_ref import rmse, psnr, uqi, ergas, scc, rase, sam, vifp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sewar: filename=sewar-0.4.5-py3-none-any.whl size=10608 sha256=45e8b09b1d14598cc591a4f885a50b1b1e6f88aef7c59cde66e4892ce02a9b77
  Stored in directory: /root/.cache/pip/wheels/7a/38/6b/d066cfcb2b1c1c9b059b9ce1bdc4803b469359214e8dad4e3e
Successfully built sewar


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim , interpolation = inter)

    # return the resized image
    return resized

def remove_black_borders(image):
    y_nonzero, x_nonzero, _ = np.nonzero(image)
    return image[np.min(y_nonzero):np.max(y_nonzero), np.min(x_nonzero):np.max(x_nonzero)]    

In [ ]:
"""
path = glob.glob("/content/drive/MyDrive/UCD_Research/LANDSAT_8_2013_2022_B5/*.TIF")
i=1
for file in path:
  img = cv2.imread(file)
  s = file.split("_")
  name = "/content/drive/MyDrive/UCD_Research/"+s[8]+".TIF"
  imgb = imutils.rotate(img,angle=14.3)
  imgc = remove_black_borders(imgb)
  imgd = cv2.resize(imgc,(1080,1920),interpolation = cv2.INTER_CUBIC)
  #imge = cv2.imwrite(name,imgd)
  #print("File Number Done : ",i)
  i+=1
"""
#a = imgc
#b = cv2.imwrite("/content/drive/MyDrive/UCD_Research/a.TIF",a)

# 11.8 degree clockwise rotation for LANDSAT 5 and 7
# 14.3 degree clockwise rotation for LANDSAT 8
# Writing Images to Google Drive is done
# Cropping the extra black space around image is done

# Image Preprocessing is done

In [ ]:
"""
otsu_threshold, image_result = cv2.threshold(imgc, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
print("Obtained threshold: ", otsu_threshold)


# Process - 1 : Convert to Grayscale
imgc[imgc>61] = 255
imgc[imgc<=61] = 0
cv2_imshow(imgc)


# Process - 2: Closing
kernel = np.ones((5, 5), np.uint8)
imgc = cv2.morphologyEx(imgc, cv2.MORPH_CLOSE, kernel)
print("Closing")
cv2_imshow(imgc)


# Process - 3 Gradient
#kernel = np.ones((30, 30), np.uint8)
grad = cv2.morphologyEx(imgc, cv2.MORPH_GRADIENT, kernel)
print("Gradient")
cv2_imshow(grad)

# Process - 4 Black Hat
#kernel = np.ones((30, 30), np.uint8)
bhat = cv2.morphologyEx(imgc, cv2.MORPH_BLACKHAT, kernel)
print("Black Hat")
cv2_imshow(bhat)

# Process - 5 Top Hat
#kernel = np.ones((30, 30), np.uint8)
that = cv2.morphologyEx(imgc, cv2.MORPH_TOPHAT, kernel)
print("Top Hat")
cv2_imshow(that)

# Process - 6 Opening
#kernel = np.ones((30, 30), np.uint8)
open = cv2.morphologyEx(imgc, cv2.MORPH_TOPHAT, kernel)
print("Opening")
cv2_imshow(open)

# Process - 7 Erosion
#kernel = np.ones((30, 30), np.uint8)
ero = cv2.morphologyEx(imgc, cv2.MORPH_TOPHAT, kernel)
print("Erosion")
cv2_imshow(ero)

# Process - 8 Dilation
#kernel = np.ones((30, 30), np.uint8)
dil = cv2.morphologyEx(imgc, cv2.MORPH_TOPHAT, kernel)
print("Dilation")
cv2_imshow(dil)


# Process - 9 Canny

can_clo = cv2.Canny(imgc,threshold1 = 100, threshold2 = 200)
cv2_imshow(can_clo)

# Process - 10 Sobel

sob_x_clo = cv2.Sobel( imgc, cv2.CV_8U, 1,0, ksize = 5)
sob_y_clo = cv2.Sobel(imgc, cv2.CV_8U, 0,1, ksize = 5)
sob_clo = sob_x_clo + sob_y_clo
cv2_imshow(sob_clo)

# Process - 11 Scharr

schx_clo = cv2.Scharr(imgc, cv2.CV_8U,1,0)
schy_clo = cv2.Scharr(imgc, cv2.CV_8U,0,1)
sch_clo = schx_clo+schy_clo
cv2_imshow(sch_clo)

# Process - 12 Prewitt

kx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
ky = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
img_prewittx = cv2.filter2D(imgc, -1, kx)
img_prewitty = cv2.filter2D(imgc, -1, ky)
pre_clo = img_prewittx + img_prewitty
cv2_imshow(pre_clo)



cv2_imshow(imga)
edges = cv2.Canny(imga,threshold1 = 50, threshold2 = 300)
cv2_imshow(edges)

img = cv2.imread("/content/L1C_T29UPV_A002408_20151208T114454.tif")
imga = cv2.resize(img,(1080,1920))
stdimg = cv2.cvtColor(imga, cv2.COLOR_BGR2GRAY)
cv2_imshow(stdimg)
edge = cv2.Canny(stdimg,threshold1 = 100, threshold2 = 200)
cv2_imshow(edge)

IMPORTANT :
Before processing the images, resize each image to uniform size. After resizing 
the image, rotate the image by 13.2 degrees to straighten up the image. Then crop
it into vertical sections (if required) and perform the edge detection or semantic
segmentation whichever seems more viable. 
"""

In [ ]:
# Code for applying all the detectors
'''
path = glob.glob("/content/drive/MyDrive/UCD_Research/Data_LANDSAT_8/*.TIF")
i=1

for file in path:
  img = cv2.imread(file)
  s = file.split("_")
  name = "/content/drive/MyDrive/UCD_Research/Edge_LANDSAT_8/Prewitt_Edge_"+s[3][2:10]+".TIF"
  print(name)
  
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  otsu_threshold, image_result = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  kernel = np.ones((10, 10), np.uint8)

  print("File Number Done : ",i)
  img[img>otsu_threshold-10] = 255
  img[img<=otsu_threshold-10] = 0
  
  img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
  kx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]]) 
  ky = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
  img_prewittx = cv2.filter2D(img, -1, kx)
  img_prewitty = cv2.filter2D(img, -1, ky)
  imgb = img_prewittx + img_prewitty
  
  imga = cv2.imwrite(name,imgb)
  i+=1
'''  

In [ ]:
#https://stackoverflow.com/questions/24768222/how-to-detect-a-shift-between-images
#https://itom.bitbucket.io/latest/docs/12_tutorials/lateralShift.html
#Cross Correlation in scikit learn
#https://dsp.stackexchange.com/questions/38065/peak-signal-to-noise-ratio-psnr-in-python-for-an-image
#https://ourcodeworld.com/articles/read/991/how-to-calculate-the-structural-similarity-index-ssim-between-two-images-with-python#:~:text=Learn%20how%20to%20determine%20the,by%20losses%20in%20data%20transmission.
# Now refer this for image differencing

In [ ]:
# Code for finding the shift between two images
"""
path = glob.glob("/content/drive/MyDrive/UCD_Research/Edge_LANDSAT_8/*.TIF")
for file in path:
  s = file.split("/")
  date_name = s[6][-12:-4]
  map_name = s[6][:-13]
  new_name = s[0]+"/"+s[1]+"/"+s[2]+"/"+s[3]+"/"+s[4]+"/"+s[5]+"/"+date_name+"_"+map_name+".TIF"
  print(new_name)
  os.rename(file,new_name)
  #img = cv2.imread(file)
"""

#ind = ['Reference_Image', 'Canny_Image', 'Sobel_Image', 'Scharr_Image', 'Prewitt_Image']
ind = []
col = ['RMSE', 'PSNR', 'UQI', 'SSIM', 'ERGAS', 'SCC', 'SAM', 'VIFP','HDF']
data=[]
i=0
path = sorted(glob.glob("/content/drive/MyDrive/UCD_Research/Edge_LANDSAT_8/*.TIF"))
for file in path:
  s = file.split("/")
  pref = s[0]+"/"+s[1]+"/"+s[2]+"/"+s[3]+"/"+s[4]+"/"+s[5]+"/"
  if s[6][9] == "R":
    ref = cv2.imread(file)
    date_name = s[6][:8]
    can_name = pref+date_name+"_Edge.TIF"
    sob_name = pref+date_name+"_Sobel_Edge.TIF"
    sch_name = pref+date_name+"_Scharr_Edge.TIF"
    pre_name = pref+date_name+"_Prewitt_Edge.TIF"
    
    can = cv2.imread(can_name)
    sob = cv2.imread(sob_name)
    sch = cv2.imread(sch_name)
    pre = cv2.imread(pre_name)

    rmse_can = rmse(ref,can)
    rmse_sob = rmse(ref,sob)
    rmse_sch = rmse(ref,sch)
    rmse_pre = rmse(ref,pre)
    rmse_ref = rmse(ref,ref)

    psnr_can = psnr(ref,can)
    psnr_sob = psnr(ref,sob)
    psnr_sch = psnr(ref,sch)
    psnr_pre = psnr(ref,pre)
    psnr_ref = psnr(ref,ref)

    uqi_can = uqi(ref,can)
    uqi_sob = uqi(ref,sob)
    uqi_sch = uqi(ref,sch)
    uqi_pre = uqi(ref,pre)
    uqi_ref = uqi(ref,ref)

    ssim_can = ssim(ref,can,multichannel=True)
    ssim_sob = ssim(ref,sob,multichannel=True)
    ssim_sch = ssim(ref,sch,multichannel=True)
    ssim_pre = ssim(ref,pre,multichannel=True)
    ssim_ref = ssim(ref,ref,multichannel=True)

    ergas_can = ergas(ref,can)
    ergas_sob = ergas(ref,sob)
    ergas_sch = ergas(ref,sch)
    ergas_pre = ergas(ref,pre)
    ergas_ref = ergas(ref,ref)

    scc_can = scc(ref,can)
    scc_sob = scc(ref,sob)
    scc_sch = scc(ref,sch)
    scc_pre = scc(ref,pre)
    scc_ref = scc(ref,ref)

    sam_can = sam(ref,can)
    sam_sob = sam(ref,sob)
    sam_sch = sam(ref,sch)
    sam_pre = sam(ref,pre)
    sam_ref = sam(ref,ref)

    vifp_can = vifp(ref,can)
    vifp_sob = vifp(ref,sob)
    vifp_sch = vifp(ref,sch)
    vifp_pre = vifp(ref,pre)
    vifp_ref = vifp(ref,ref)

    """
    msssim_can = msssim(ref,can)
    msssim_sob = msssim(ref,sob)
    msssim_sch = msssim(ref,sch)
    msssim_pre = msssim(ref,pre)
    msssim_ref = msssim(ref,ref)
    """
    hdf_can = hdf(ref,can)
    hdf_sob = hdf(ref,ref)
    hdf_sch = hdf(ref,sch)
    hdf_pre = hdf(ref,pre)
    hdf_ref = hdf(ref,ref)

    row_a = [rmse_ref,psnr_ref,uqi_ref,ssim_ref,ergas_ref,scc_ref,sam_ref,vifp_ref,hdf_ref]
    row_b = [rmse_can,psnr_can,uqi_can,ssim_can,ergas_can,scc_can,sam_can,vifp_can,hdf_can]
    row_c = [rmse_sob,psnr_sob,uqi_sob,ssim_sob,ergas_sob,scc_sob,sam_sob,vifp_sob,hdf_sob]
    row_d = [rmse_sch,psnr_sch,uqi_sch,ssim_sch,ergas_sch,scc_sch,sam_sch,vifp_sch,hdf_sch]
    row_e = [rmse_pre,psnr_pre,uqi_pre,ssim_pre,ergas_pre,scc_pre,sam_pre,vifp_pre,hdf_pre]
    sp = [" "," "," "," "," "," "," "," "," "]

    ref_name = 'Reference - ' + date_name
    ind.append([ref_name, 'Canny_Image', 'Sobel_Image', 'Scharr_Image', 'Prewitt_Image','_'])
    data.append(row_a)
    data.append(row_b)
    data.append(row_c)
    data.append(row_d)
    data.append(row_e)
    data.append(sp)
    i+=1
    print("Date done: ",date_name)
    
from itertools import chain
flat_ind = list(chain.from_iterable(ind))
df1 = pd.DataFrame(data,index=flat_ind,columns=col)
df1.to_excel("/content/drive/MyDrive/UCD_Research/LANDSAT_8_Edge_Metric.xlsx")



KeyboardInterrupt: ignored

In [ ]:
"""
from itertools import chain
#print(ind)
flat_ind = list(chain.from_iterable(ind))
print(flat_ind)
df1 = pd.DataFrame(data,index=flat_ind,columns=col)
df1.to_excel("/content/drive/MyDrive/UCD_Research/LANDSAT_8_Edge_Metric.xlsx")
"""    

['Reference - 20010305', 'Canny_Image', 'Sobel_Image', 'Scharr_Image', 'Prewitt_Image', '_', 'Reference - 20010508', 'Canny_Image', 'Sobel_Image', 'Scharr_Image', 'Prewitt_Image', '_', 'Reference - 20010524', 'Canny_Image', 'Sobel_Image', 'Scharr_Image', 'Prewitt_Image', '_', 'Reference - 20010828', 'Canny_Image', 'Sobel_Image', 'Scharr_Image', 'Prewitt_Image', '_', 'Reference - 20011031', 'Canny_Image', 'Sobel_Image', 'Scharr_Image', 'Prewitt_Image', '_', 'Reference - 20020119', 'Canny_Image', 'Sobel_Image', 'Scharr_Image', 'Prewitt_Image', '_', 'Reference - 20021018', 'Canny_Image', 'Sobel_Image', 'Scharr_Image', 'Prewitt_Image', '_', 'Reference - 20021103', 'Canny_Image', 'Sobel_Image', 'Scharr_Image', 'Prewitt_Image', '_', 'Reference - 20030327', 'Canny_Image', 'Sobel_Image', 'Scharr_Image', 'Prewitt_Image', '_', 'Reference - 20030530', 'Canny_Image', 'Sobel_Image', 'Scharr_Image', 'Prewitt_Image', '_', 'Reference - 20031021', 'Canny_Image', 'Sobel_Image', 'Scharr_Image', 'Prewitt_

In [ ]:
"""
New table rough structure:
Dates --->
Canny Sobel |^

index_error_5 = []
path = sorted(glob.glob("/content/drive/MyDrive/UCD_Research/Edge_LANDSAT_5/*.TIF"))
for file in path:
  s = file.split("/")
  if s[6][9] == "R":
    date_name = s[6][:8]
    index_error.append(date_name)
"""
!pip install image_similarity_measures
import image_similarity_measures
from image_similarity_measures.quality_metrics import fsim, issm, sre


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 1.2 MB/s 
     |████████████████████████████████| 1.7 MB 39.8 MB/s 
  Created wheel for phasepack: filename=phasepack-1.5-py3-none-any.whl size=26984 sha256=c54b8c3af7d078e0647f65b167993640a2fd6109b1016af573f83122b7823735
  Stored in directory: /root/.cache/pip/wheels/39/5c/b3/79292d7ac3f088e858805b5515b45cf5a365106397d936caf1
Successfully built phasepack


In [ ]:
ind = ["Canny_Filter","Sobel_Filter","Scharr_Filter","Prewitt_Filter"]
col=[]
data=[]
df1 = pd.DataFrame(data,index=ind,columns=col)


In [ ]:
i=0
path = sorted(glob.glob("/content/drive/MyDrive/UCD_Research/Edge_LANDSAT_5/*.TIF"))
for file in path:
  s = file.split("/")
  pref = s[0]+"/"+s[1]+"/"+s[2]+"/"+s[3]+"/"+s[4]+"/"+s[5]+"/"
  if s[6][9] == "R":
    ref = cv2.imread(file)
    date_name = s[6][:8]
    can_name = pref+date_name+"_Edge.TIF"
    sob_name = pref+date_name+"_Sobel_Edge.TIF"
    sch_name = pref+date_name+"_Scharr_Edge.TIF"
    pre_name = pref+date_name+"_Prewitt_Edge.TIF"
    
    can = cv2.imread(can_name)
    sob = cv2.imread(sob_name)
    sch = cv2.imread(sch_name)
    pre = cv2.imread(pre_name)

    rmse_can = rase(ref,can)
    rmse_sob = rase(ref,sob)
    rmse_sch = rase(ref,sch)
    rmse_pre = rase(ref,pre)
    rm = [rmse_can,rmse_sob,rmse_sch,rmse_pre]

    df1.insert(i,date_name,rm,True)
    print("Date done :",date_name)
    i+=1

print(df1)

/usr/local/lib/python3.7/dist-packages/sewar/full_ref.py:248: RuntimeWarning: divide by zero encountered in true_divide
  rase_map = (100./M) * np.sqrt( np.sum(rmse_map**2,axis=2) / N )
/usr/local/lib/python3.7/dist-packages/sewar/full_ref.py:248: RuntimeWarning: invalid value encountered in multiply
  rase_map = (100./M) * np.sqrt( np.sum(rmse_map**2,axis=2) / N )


Date done : 19840704
Date done : 19841125
Date done : 19850418
Date done : 19860912
Date done : 19861115
Date done : 19880613
Date done : 19890702
Date done : 19890718
Date done : 19900721
Date done : 19920523
Date done : 19921030
Date done : 19930627
Date done : 19940326
Date done : 19940902
Date done : 19941004
Date done : 19941105
Date done : 19950225
Date done : 19950414
Date done : 19950804
Date done : 19950820
Date done : 19960228
Date done : 19960619
Date done : 19970910
Date done : 19971113
Date done : 19980201
Date done : 19990916
Date done : 19991119
Date done : 20001020
Date done : 20040218
Date done : 20040305
Date done : 20051119
Date done : 20060428
Date done : 20060717
Date done : 20061106
Date done : 20070501
Date done : 20070922
Date done : 20090420
Date done : 20090911
Date done : 20100525
Date done : 20101016
Date done : 20110325
                19840704  19841125  19850418  19860912  19861115  19880613  \
Canny_Filter         NaN       NaN       NaN       NaN       

In [ ]:
df1.to_excel("/content/drive/MyDrive/UCD_Research/RASE.xlsx")